In [1]:
import sys
import os
import json

path='C:/Users/jeffs/Desktop/M2/'
sys.path.append(os.path.join(path , 'Transformer'))

from melodygenerator import *
from melodypreprocessor import *
from train import *
from transformer import *
from train import _train_step

In [2]:
with open('Data/data.json','r') as file: #le fichier crée par extract_data.py
    data= json.load(file)

In [3]:
QUARTER_DURATION= { '32nd': '0.125', '16th' : '0.25' , 'eighth' : '0.5' ,'quarter' : '1.0' ,'half' : '2.0'}
dataset = []    
for P in data:
    parts=[]
    for name_part in ['A','B','C','D']:
        if name_part in P:
            parts.append(name_part)
    for part in parts:
        all_notes=""
        for k in P[part].keys():
            if k!='key':
                notes=P[part][k]['Notes']
                for note in notes:
                    all_notes+=note[0]+"-"+QUARTER_DURATION[note[1]]+', '
        if len(all_notes)!=0:
            dataset.append(all_notes[:-2])
            if len(dataset)==20:                #j'ai mis 20 melody pour tester car m'apprentissage peut devenir long
                with open('Data/dataset.json','w') as file:
                    json.dump(dataset,file) 
                break

           
    
    

In [4]:
melody_preprocessor = MelodyPreprocessor("Data/dataset.json", batch_size=32)
train_dataset = melody_preprocessor.create_training_dataset()
vocab_size = melody_preprocessor.number_of_tokens_with_padding
transformer_model = Transformer(
    num_layers=2,
    d_model=64,
    num_heads=2,
    d_feedforward=128,
    input_vocab_size=vocab_size,
    target_vocab_size=vocab_size,
    max_num_positions_in_pe_encoder=MAX_POSITIONS_IN_POSITIONAL_ENCODING,
    max_num_positions_in_pe_decoder=MAX_POSITIONS_IN_POSITIONAL_ENCODING,
    dropout_rate=0.1,
)
train(train_dataset, transformer_model, 10)



Training the model...
Epoch 1 Batch 1 Loss 4.516432762145996
Epoch 1 Batch 2 Loss 3.6300137042999268
Epoch 1 Batch 3 Loss 3.5128397941589355
Epoch 1 Batch 4 Loss 3.1138784885406494
Epoch 1 Batch 5 Loss 3.319992780685425
Epoch 1 Batch 6 Loss 3.283601760864258
Epoch 1 Batch 7 Loss 3.3460302352905273
Epoch 1 Batch 8 Loss 3.1845510005950928
Epoch 1 Batch 9 Loss 3.135364055633545
Epoch 1 Batch 10 Loss 3.2737863063812256
Epoch 1 Batch 11 Loss 3.4307138919830322
Epoch 1 Batch 12 Loss 3.3739476203918457
Epoch 1 Batch 13 Loss 3.9938137531280518
Epoch 1 Batch 14 Loss 4.0275959968566895
Epoch 1 Batch 15 Loss 3.3247671127319336
Epoch 1 Batch 16 Loss 3.408989667892456
Epoch 1 Batch 17 Loss 3.303338050842285
Epoch 1 Batch 18 Loss 3.281675100326538
Epoch 1 Batch 19 Loss 3.2932870388031006
Epoch 1 Batch 20 Loss 3.2565054893493652
Epoch 1 Batch 21 Loss 3.0547823905944824
Epoch 1 Batch 22 Loss 2.9119200706481934
Epoch 1 Batch 23 Loss 2.9229509830474854
Epoch 1 Batch 24 Loss 2.888157844543457
Epoch 1 Bat

In [5]:
#transformer_model.save("transformer_trained_1")

In [6]:
#transformer_trained= tf.keras.models.load_model("transformer_trained_1")

In [ ]:
print("Generating a melody...")
melody_generator = MelodyGenerator(
    transformer_model, melody_preprocessor.tokenizer,100
)
start_sequence = ["C#6-0.25", "C6-0.25", "B5-0.25", "A5-0.25", "F#6-0.25", "E6-0.5", "F#5-0.25", "C#6-0.25", "C6-0.25", "B5-0.25", "A5-0.25"]
#start_sequence = ["E6-0.5", "G5-0.5"]

In [ ]:
new_melody = melody_generator.generate(start_sequence)
print(f"Generated melody: {new_melody}")

## Visualisation

In [9]:
import music21
from music21 import *

In [10]:
us=music21.environment.UserSettings()
us.restoreDefaults()

In [11]:
#Morgan (Windows)
music21.environment.set('musescoreDirectPNGPath',str(os.path.join("C:\\", "Program Files","MuseScore 4","bin","MuseScore4.exe")))
music21.environment.set('musicxmlPath', str(os.path.join("C:\\", "Program Files","MuseScore 4","bin","MuseScore4.exe")))
music21.environment.set('lilypondPath', r'C:\Users\jeffs\lilypond-2.24.2\bin\lilypond.exe')

In [ ]:
def visualize_melody(melody):
    """
    Visualize a sequence of (pitch, duration) pairs using music21.

    Parameters:
        - melody (list): A list of (pitch, duration) pairs.
    """
    score = stream.Score()
    score.metadata = metadata.Metadata(title="Melody generated")
    score.metadata.keySignature= key.Key("F")
    part = stream.Part()
    part.append(meter.TimeSignature("2/4"))
    for n in melody.split():
        splitted=n.split('-')
        if len(splitted)==2:
            note,duration=splitted[0],splitted[1]
        else:
            note = "".join( substring for substring in splitted[:-1])
            duration=splitted[-1]
        part.append(music21.note.Note(note,quarterLength=float(duration)))
    score.append(part)
    score.show('text')
    score.show('musicxml')
    
    
visualize_melody(new_melody)

{0.0} <music21.metadata.Metadata object at 0x20346d0b508>
{0.0} <music21.stream.Part 0x20346e42cc8>
    {0.0} <music21.meter.TimeSignature 2/4>
    {0.0} <music21.note.Note E>
    {0.5} <music21.note.Note G>
    {1.0} <music21.note.Note F>
    {1.5} <music21.note.Note D>
    {2.0} <music21.note.Note B>
    {2.5} <music21.note.Note A>
    {3.0} <music21.note.Note D>
    {3.5} <music21.note.Note G>
    {4.0} <music21.note.Note F>
    {4.5} <music21.note.Note D>
    {5.0} <music21.note.Note G>
    {5.5} <music21.note.Note F>
    {6.0} <music21.note.Note D>
    {6.5} <music21.note.Note G>
    {7.0} <music21.note.Note F>
    {7.5} <music21.note.Note D>
    {8.0} <music21.note.Note G>
    {8.5} <music21.note.Note F>
    {9.0} <music21.note.Note D>
    {9.5} <music21.note.Note G>
    {10.0} <music21.note.Note F>
    {10.5} <music21.note.Note D>
    {11.0} <music21.note.Note B>
    {11.5} <music21.note.Note A>
    {12.0} <music21.note.Note D>
    {12.5} <music21.note.Note G>
    {13.0} <music2